In [80]:
import ctypes
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [81]:
class MapsForgeHeader(ctypes.BigEndianStructure):
    _pack_ = 1
    _fields_ = [
        ("magic_byte", ctypes.c_char * 20),
        ("header_size", ctypes.c_uint32),
        ("file_version", ctypes.c_uint32),
        ("file_size", ctypes.c_uint64),
        ("date_of_creation", ctypes.c_uint64),
        ("minLat", ctypes.c_int32),
        ("minLon", ctypes.c_int32),
        ("maxLat", ctypes.c_int32),  # most nothern point
        ("maxLon", ctypes.c_int32),
        ("tile_size", ctypes.c_uint16),
    ]


class ParsedFileName:
    country: str
    state: str
    creationDate: str
    minLong: int
    minLat: int
    sizeNorthSouth: int
    sizeWestEast: int

    def __init__(self, filename):
        self.country = filename[:2]
        self.state = filename[2:6]
        self.creationDate = filename[6:12]
        self.field1 = int(filename[12:15], 36)
        self.field2 = int(filename[15:18], 36)
        self.z1 = int(filename[18:21], 36)
        self.z2 = int(filename[21:24], 36)


def parseHeader(file: Path) -> MapsForgeHeader:
    header = MapsForgeHeader.from_buffer_copy(file.read_bytes())
    return header


def parseFilename(filename) -> ParsedFileName:
    return ParsedFileName(filename)

In [82]:
files = [f for f in Path("./allmaps").glob(("*.map")) if f.is_file()]
data = []


print("--- files ----   (", len(files), ")")

for i in files:
    print(i.name)

--- files ----   ( 253 )
asia_Bangladesh-BD00002206254PE2OA02Z05J.map
asia_Armenia-AM00002206253X82D2021021.map
asia_Kuwait-KW00002303103Z62LS01L017.map
asia_Iran-IR00002206253XL2EB0C80BT.map
asia_Turkmenistan-TM000022062542J2BQ09E06B.map
asia_Tajikistan-TJ00002206254CB2D904Z03L.map
asia_Afghanistan-AF00002311114802FD09406Z.map
asia_Bahrain-BH000023031041J2O900G00U.map
asia_Brunei-BN000023031055V32J00U00Q.map
asia_Cambodia-KH00002303104YG2WD03D03G.map
asia_China_Beijing-CN020025040255B2D902R04Q.map
asia_China_Anhui-CN010025040256C2ID039045.map
asia_China_Chongqing-CN030025040250B2K903N02Y.map
asia_China_Fujian-CN040025040256Y2MZ03503H.map
asia_China_Gansu-CN05002504024RV2BA0AV08Q.map
asia_China_Guangdong-CN06002504025332OT04U041.map
asia_China_Hainan-CN090025040252F2SP02M02Z.map
asia_China_Hebei-CN100025040255B2BT049066.map
asia_China_Guizhou-CN08002504024Z82MD03V03F.map
asia_China_Hongkong-CN130025040255Q2R400E00A.map
asia_China_Henan-CN12002504025352EJ04J06C.map
asia_China_Guangxi-CN

In [83]:
for f in files:
    header = parseHeader(f)
    # fix modified name (my maps are named like Vorarlberg-AT08002303103BS27H00H00P.map)
    humanName, filename = f.name.split("-")
    pf = parseFilename(filename)
    data.append({"file": f, "header": header, "parsedFilename": pf, "Name": humanName})

In [84]:
names = [d["Name"] for d in data]
minLat = [d["header"].minLat for d in data]
minLon = [d["header"].minLon for d in data]
maxLat = [d["header"].maxLat for d in data]
maxLon = [d["header"].maxLon for d in data]

f1 = [d["parsedFilename"].field1 for d in data]
f2 = [d["parsedFilename"].field2 for d in data]
f3 = [d["parsedFilename"].z1 for d in data]
f4 = [d["parsedFilename"].z2 for d in data]


df = pd.DataFrame(
    {
        "name": names,
        "minLat": minLat,
        "minLon": minLon,
        "maxLat": maxLat,
        "maxLon": maxLon,
        "f1": f1,
        "f2": f2,
        "f3": f3,
        "f4": f4,
    }
)
df["minLat"] = df["minLat"] / 1000000
df["minLon"] = df["minLon"] / 1000000
df["maxLat"] = df["maxLat"] / 1000000
df["maxLon"] = df["maxLon"] / 1000000

df["angleDiffLongitude"] = df["maxLon"] - df["minLon"]
df["angleDiffLatgitude"] = df["maxLat"] - df["minLat"]

df['continent'] = df['name'].str.split('_',n=1, expand=True)[0]

display(df)

,name,minLat,minLon,maxLat,maxLon,f1,f2,f3,f4,angleDiffLongitude,angleDiffLatgitude,continent
0,asia_Bangladesh,18.58403,87.997650,26.64677,92.69714,6098,3466,107,199,4.699490,8.06274,asia
1,asia_Armenia,38.83743,43.438210,41.31029,46.64262,5084,3062,73,73,3.204410,2.47286,asia
2,asia_Kuwait,28.49500,46.535000,30.12500,49.02500,5154,3376,57,43,2.490000,1.63000,asia
3,asia_Iran,24.03947,44.023030,39.79045,63.35413,5097,3107,440,425,19.331100,15.75098,asia
4,asia_Turkmenistan,35.12355,51.832710,42.86081,66.70892,5275,3014,338,227,14.876210,7.73726,asia
...,...,...,...,...,...,...,...,...,...,...,...,...
248,europe_Germany_CityOfBerlin,52.33446,13.082830,52.67830,13.76225,4393,2680,16,13,0.679420,0.34384,europe
249,europe_Germany_HamburgCity,53.38581,9.613465,53.74650,10.33637,4314,2640,17,13,0.722905,0.36069,europe
250,europe_Germany_Hesse,49.39321,7.768021,51.65916,10.24595,4272,2718,57,81,2.477929,2.26595,europe
251,europe_Germany_LowerSaxony,51.29348,6.295338,54.07551,11.60274,4239,2627,121,104,5.307402,2.78203,europe


In [85]:
fig = px.scatter(
    df,
    x="maxLon",
    y="f1",
    color="continent",
    title="Value marked with <> dependend on Longitude AT0000111111 <3BR> 262 04W 02L in Base36",
    hover_name="name",
    hover_data=["maxLon", "f1"],
    labels={
        "f1": "Value from filename",
        "maxLon": "Longitude in Degrees",
    },
)
fig.write_html("docs/f1-maxLon.html")
fig.show()

In [ ]:
fig = px.scatter(
    df,
    x="maxLat",
    y="f2",
    color="continent",
    title="Value marked with <> dependend on Latitude AT0000111111 3BR <262> 04W 02L in Base36",
    hover_name="name",
    hover_data=["maxLat", "f2"],
    labels={
        "f2": "Value from filename",
        "maxLat": "Latitude in Degrees",
    },
)
fig.write_html("docs/f2-maxLat.html")
fig.show()

In [ ]:
fig = px.scatter(
    df,
    x="angleDiffLongitude",
    y="f3",
    color="continent",
    title="Value marked with <> dependend on Longitude difference AT0000111111 3BR 262 <04W> 02L in Base36",
    hover_name="name",
    hover_data=["angleDiffLongitude", "f3"],
    labels={
        "f3": "Value from filename",
        "angleDiffLongitude": "Longitude in Degrees",
    },
)
fig.write_html("docs/f3-angleDiffLongitude.html")
fig.show()

In [ ]:
fig = px.scatter(
    df,
    x="angleDiffLatgitude",
    y="f4",
    color="continent",
    title="Value marked with <> dependend on Latitude difference AT0000111111 3BR 262 04W <02L> in Base36",
    hover_name="name",
    hover_data=["angleDiffLatgitude", "f4"],
    labels={
        "f4": "Value from filename",
        "angleDiffLatgitude": "Latitude in Degrees",
    },
)
fig.write_html("docs/f4-angleDiffLatgitude.html")
fig.show()